In [2]:
import numpy as np
import numpy
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import pyscf.lib
from pyscf.cc import CCSD
from pyscf.data import nist
import matplotlib.pyplot as plt
import basis_set_exchange as bse
angstrom = 1 / 0.52917721067

In [2]:
COmol = pyscf.gto.Mole(atom = 'C 0 0 0.; O 0 0 2.',basis='6-31G',unit='au')
COmol.build()
COHF = pyscf.scf.RHF(COmol)

In [3]:
def Alch_calc(calc,deltaZ):
    mf=pyscf.qmmm.mm_charge(calc,calc.mol.atom_coords()/angstrom,deltaZ)
    includeonly = [0, 1]
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):            
            q = mf.mol.atom_charges().astype(np.float)
            q[includeonly] -= deltaZ
            return mf.mol.energy_nuc(q)
    mf = NoSelfQMMM()
    return mf

In [4]:
def Alch_e_rdm1(calc,deltaZ):
    mf=Alch_calc(calc,deltaZ)
    e=mf.kernel()
    dm1_ao=mf.make_rdm1()
    return [e,dm1_ao]

In [13]:
COdm=Alch_e_rdm1(COHF,np.array((0.,0.)))[1]

converged SCF energy = -112.652239281562


In [9]:
"""returns the delta V matrix in ao basis for the alchemical perturbation"""
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1]/angstrom)
    dV+=mol.intor('int1e_rinv')*dL[1]
    return dV    

In [41]:
# Target molecule NN@CO 
Alch_calc(COHF,np.array((-1.,1.))).kernel()-COHF.kernel()

converged SCF energy = -105.911201761195
converged SCF energy = -112.652239281562


6.741037520366547

In [42]:
np.einsum('ij,ji',COdm,DeltaV(COmol,np.array((1.,-1.))))

6.723205319249292

In [17]:
e_dm=[]
for i in np.linspace(-.2,.2,5):
    e_dm.append(Alch_e_rdm1(COHF,np.array((-i,i))))

converged SCF energy = -114.233704023131
converged SCF energy = -113.433786714567
converged SCF energy = -112.652239281562
converged SCF energy = -111.889199239812
converged SCF energy = -111.144854160312


In [18]:
es=np.asarray([e_dm[x][0] for x in range(len(e_dm))])
dms=np.asarray([e_dm[x][1] for x in range(len(e_dm))])
dms_array=np.asarray(dms)

In [33]:
coeff5p=np.asarray([[1/12,-2/3,0,2/3,-1/12],[-1/12,4/3,-5/2,4/3,-1/12],[-1/2,1,0,-1,1/2],[1,-4,6,-4,1]]) #f',f",f^3,f^4
def tayl_exp_5p(pts,dh,dL):
    pred=pts[2]
    for i in range(4):
        pred+=np.sum(pts*coeff5p[i])/np.math.factorial(i+1)*(dL/dh)**(i+1)
    return (pred)
def rho_tilda(ptsIN,dh,dL):
    pts=ptsIN.copy()
    pred=pts[2]
    for i in range(4):
        deriv=np.zeros_like(pred)
        for j in range(5):
            deriv+=(pts[j]*coeff5p[i][j])
        pred+=deriv/np.math.factorial(i+2)*(dL/dh)**(i)
    return (pred)

In [34]:
pts=dms
deriv=np.zeros_like(pts[2])
for j in range(5):
    deriv+=(pts[j]*coeff5p[3][j])
pts[1].shape

(18, 18)

In [1]:
np.einsum('ij,ji',rho_tilda(pts,.1,1.),DeltaV(COmol,np.array((1.,-1.))))

NameError: name 'np' is not defined